In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mikoaro/distracteddriver")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/mikoaro/distracteddriver/versions/1


In [ ]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split


In [ ]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
warnings.filterwarnings('ignore')

In [ ]:
train_path='/root/.cache/kagglehub/datasets/mikoaro/distracteddriver/versions/1/distracted-driver-detection/train'
test_path='/root/.cache/kagglehub/datasets/mikoaro/distracteddriver/versions/1/distracted-driver-detection/test'

In [ ]:
# Parameters
IMG_HEIGHT = 480
IMG_WIDTH = 480
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 10
TRAIN_DIR = train_path
TEST_DIR = test_path
split_ratio=0.2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),  #for increasing diversity
    transforms.RandomRotation(15),
    transforms.ToTensor(),      #converting to tensor data
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])


In [ ]:
images = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transform)

In [ ]:
images.classes

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [ ]:
images.class_to_idx

{'c0': 0,
 'c1': 1,
 'c2': 2,
 'c3': 3,
 'c4': 4,
 'c5': 5,
 'c6': 6,
 'c7': 7,
 'c8': 8,
 'c9': 9}

In [ ]:
type(images)

torchvision.datasets.folder.ImageFolder

In [ ]:
images[0]

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

In [ ]:
len(images)

22424

In [ ]:

num_total = len(images)
num_val = int(split_ratio* num_total)
num_train = num_total - num_val

# Split the dataset into training and validation sets
train_data, test_data = random_split(images, [num_train, num_val])




In [ ]:

print(f"Number of training samples: {len(train_data)}")
print(f"Number of validation samples: {len(test_data)}")

Number of training samples: 17940
Number of validation samples: 4484


In [ ]:
# DataLoaders for training and validation
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def weights_init(model, type_of_init='zero'):
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            if type_of_init == 'zero':
                nn.init.constant_(layer.weight, 0)
            elif type_of_init == 'random':
                nn.init.normal_(layer.weight, mean=0, std=0.01)
            elif type_of_init == 'he':
                nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')

In [ ]:
class CNN(nn.Module):
    def __init__(self,dropout_prob=0.2, active_fun='RELU'):
        super(CNN, self).__init__()

        # activation function
        if active_fun == 'TANH':
            self.active_fun = nn.Tanh()
        elif active_fun == 'RELU':
            self.active_fun = nn.ReLU()

        self.last_lay_activation = nn.Softmax(dim=1)


        self.block_1 = nn.Sequential(
            #convolutional layer
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            self.active_fun,
            #max ooling layer
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block_2 = nn.Sequential(
            #convolutional layer
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            self.active_fun,
            #max ooling layer
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block_3 = nn.Sequential(
            #convolutional layer
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            self.active_fun,
            #max Pooling layer
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.dropout_conv = nn.Dropout(dropout_prob)
        self.dropout_fc = nn.Dropout(dropout_prob)
        # Fully Connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            self.dropout_fc,
            nn.Linear(32 * (IMG_HEIGHT // 8) * (IMG_WIDTH // 8), 256),
            self.active_fun,
            self.dropout_fc,
            nn.Linear(256, 10),
            self.last_lay_activation
        )


    def forward(self, x):
        x = self.block_1(x)
        #x = self.block_2(x)
        #x = self.block_3(x)

        x = self.fc(x)
        return x

In [ ]:
model = CNN().to(DEVICE)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
def train_model(model, train_loader, test_loader, epochs=5, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        #setting the model to training mode
        model.train()
        #loss of current epoch
        curr_loss = 0
        correct_pred=0
        total_pred=0

        for images, classes in tqdm(train_loader, desc="Training", leave=False):
            #clearing the gradients
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, classes )
            loss.backward()
            optimizer.step()

            curr_loss += loss.item()

            #the maximum value along the label dimension
            value, predicted = torch.max(outputs, 1)
            total_pred += classes .size(0)
            correct_pred += (predicted == classes).sum().item()

        train_accuracy = 100 * correct_pred / total_pred
        avg_loss=curr_loss/len(train_loader)

        print(f'Epoch {epoch+1}, Average Loss: {avg_loss}, Accuracy: {train_accuracy}')

        # evaluation mode
        model.eval()
        #disable grad computation
        with torch.no_grad():

            correct_test_pred=0
            total_test_pred=0
            test_loss = 0

            for images, classes in test_loader:
                outputs = model(images)
                test_loss += criterion(outputs, classes).item()
                value, predicted = torch.max(outputs, 1)
                total_test_pred += classes.size(0)
                correct_test_pred += (predicted == classes).sum().item()

            test_accuracy = 100 * correct_test_pred / total_test_pred

            print(f'Validation Accuracy: {test_accuracy}')

In [ ]:
train_model(model,train_loader,test_loader)

Epoch 1, Average Loss: 2.09824329179026, Accuracy: 35.68561872909699
Validation Accuracy: 45.45049063336307


Epoch 2, Average Loss: 1.9560919373218175, Accuracy: 50.17837235228539
Validation Accuracy: 58.38537020517395


Epoch 3, Average Loss: 1.8681234788554661, Accuracy: 59.1025641025641
Validation Accuracy: 63.60392506690455


Epoch 4, Average Loss: 1.8020870487957714, Accuracy: 65.80267558528428
Validation Accuracy: 69.7814451382694


Epoch 5, Average Loss: 1.7647071732556756, Accuracy: 69.45373467112597
Validation Accuracy: 74.06333630686886
